# Run Crystal Explorer from QCrBox

## Import and setting up folders / paths

In [1]:
import shutil

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper

We create a path to run our Crystal Explorer example and get our paths from our own computer's filesystem and the internal filesystem of the QCrBox containers.

Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, use the `__init__` method instead by defining the path to the shared directory explicitely in `path_to_shared_dir` and replacing the next four lines with:

```python
pathhelper = QCrBoxPathHelper(
    path_to_shared_dir,
    'examples_crystal_explorer'
)
```

In [2]:
pathhelper = QCrBoxPathHelper.from_dotenv(
    '.env.dev',
    'examples_crystal_explorer'
)

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

## Connecting to QCrBox

We can connect to the QCrBox Inventory via python after we have started everything with `qcb up`. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [3]:
qcrbox = QCrBoxWrapper.from_server_addr('127.0.0.1', 11000)

We should see `Crystalexplorer in our application_dict and should put it into a variable.

In [4]:
qcrbox.application_dict


2024-08-22T15:59:01.031311Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


{'Crystal Explorer': <Crystal Explorer>}

In [5]:
crystal_explorer = qcrbox.application_dict['Crystal Explorer']


2024-08-22T15:59:01.042672Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


## Running the Crystal Explorer GUI in QCrBox

We can now access the GUI of Crystal Explorer within the container. To run the command needs a unified cif, which is then internally converted into the format CrystalExplorer needs. For more read the Olex2 Example Notebook (TODO Write and link a more complete explanation in docs). 

Once we run the `interactive` command, two things happen: Firstly, a new browser window with the GUI opens up. Secondly, you should see an input prompt in your execution engine for this notebook. (In Jupyter this should be at the end of the cell, in VSCode it is at the top). By pressing enter within this prompt we tell QCrBox that we are done. There is no back conversion, as we do not expect Crystal Explorer to modify the cif file. Note that this input prompt is a stopgap for an actual UI, so this feels a bit clunky at the moment.

In [6]:
shutil.copy('./input_files/input.cif', path_local / 'input.cif')

session = crystal_explorer.interactive_session(
    input_cif_path=path_qcrbox / 'input.cif',
    work_cif_path=path_qcrbox / 'work.cif',
)
session.start()

2024-08-22T15:59:01.057087Z [debug    ] Starting interactive session for 'crystal-explorer' extra={}
2024-08-22T15:59:01.437163Z [debug    ] self.prepare_calculation=<QCrBoxCalculation: 'qcrbox_calc_0xdb65c158a74949c68d6e3c54e9cbc53c'> extra={}
2024-08-22T15:59:01.455151Z [debug    ] self.run_calculation=<QCrBoxCalculation: 'qcrbox_calc_0x65cb57f16a0641089305f83724e3ac37'> extra={}


gio: http://127.0.0.1/gui/crystal-explorer: Operation not supported
